In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib
from matplotlib import pyplot
pyplot.ion()
pyplot.style.use('dark_background')

from rl.algs.dqn import *

In [36]:
# GridWorld (debugging)
from rl.algs import experiment
from rl.algs import model
from rl.algs.envs import grid_world

BATCH_SIZE = 32
TRAIN_START_STEP = 3 * BATCH_SIZE
NUM_TRAIN_STEPS = 10000 # 17
REPLAY_SIZE = 100000
TARGET_NETWORK_UPDATE_FREQ = 500


env = environment.GridWorldWrapped(grid_shape=(4, 4, 1))
conv_specs = {
  model.ConvSpec(4, 1, 32),
}
fc_specs = (64,)
model_params = {
  'conv_specs': conv_specs,
  'fc_specs': fc_specs,
}

EPS_DECAY = util.Schedule((0, TRAIN_START_STEP, NUM_TRAIN_STEPS / 1, NUM_TRAIN_STEPS),
                          (1, 1, 0.0, 0.0))
EPS_RANDOM = util.Schedule((0,), (1.0,))
policy_params = {
    'lr': 0.005,
    'eps_sched': EPS_DECAY,  # TODO: change
    'target_update_freq': TARGET_NETWORK_UPDATE_FREQ,
}
exp = experiment.Experiment(env, model.QNetwork, model_params,
                            Dqn, policy_params)

print(exp.policy.model)

rb = ReplayBuffer(REPLAY_SIZE)
for step in range(TRAIN_START_STEP):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)

eps_rew = 0
for step in range(TRAIN_START_STEP, NUM_TRAIN_STEPS):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)
    
  experience_batch = rb.sample(BATCH_SIZE)
  m = exp.policy.update(experience_batch)
  
  eps_rew += sars.r
  if sars.s1 is None:
    m['r_per_eps'] = eps_rew
    eps_rew = 0
  if step % 100 == 0 or sars.s1 is None:
    for k, v in m.items():
      exp.plt.add_data(k, step, v)
      
# Expect about r_per_eps=-50 in pure random agent.

(1, 1, 32)
QNetwork(
  (convs): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=4, bias=True)
  )
)


In [11]:
opt = grid_world.GridWorld((4, 4)).optimal_expected_reward()
print(f'optimal expected r: {opt}')

optimal expected r: 1.6666666666666665


In [37]:
env.show_model_policy(exp.policy.model)

[[ 0.          0.00459319 -0.93927145 -1.70513606]
 [-0.07293212 -0.8611213  -1.76077914 -2.87749481]
 [-0.86544073 -1.88984895 -2.81150746 -3.85319376]
 [-1.9476409  -2.9053762  -3.51383066 -4.31065369]]
[['⟲' '←' '←' '←']
 ['↑' '↑' '↑' '↑']
 ['↑' '↑' '↑' '←']
 ['↑' '↑' '↑' '↓']]
[[ 0.05699685  0.         -0.02683014 -1.06846642]
 [-0.81819499  0.11112297 -0.79165423 -1.81324887]
 [-1.87141514 -0.93451715 -1.75177217 -2.85440207]
 [-2.94600296 -2.02592707 -2.73663783 -3.6263504 ]]
[['→' '⟲' '←' '←']
 ['→' '↑' '←' '←']
 ['↑' '↑' '↑' '←']
 ['↑' '↑' '↑' '↑']]
[[-1.00354743 -0.03162104  0.          0.05031908]
 [-1.86413097 -0.80993354  0.09551504 -0.93800461]
 [-2.79857254 -1.92101598 -1.05296493 -1.94411707]
 [-3.63925099 -2.89192867 -2.1466794  -3.01724148]]
[['→' '→' '⟲' '←']
 ['↑' '↑' '↑' '↑']
 ['↑' '↑' '↑' '↑']
 ['↑' '↑' '↑' '↑']]
[[-1.65495479 -0.79342598  0.03252473  0.        ]
 [-2.57044339 -1.59389126 -0.89238834 -0.03041527]
 [-3.41909337 -2.48572683 -1.86717129 -0.99387527]
 

In [38]:
# Debug the action-value function for a given state.
s = [[0, 0, 0, 0],
     [0, 0, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 2, 0]]
s = np.expand_dims(np.expand_dims(s, 0), -1)
print(exp.policy.model(util.to_variable(s)))
print(exp.policy.target_model(util.to_variable(s)))

Variable containing:
-2.1745 -1.8925 -0.0580 -1.6959
[torch.cuda.FloatTensor of size 1x4 (GPU 0)]

Variable containing:
-2.0959 -2.1576 -0.1639 -1.9269
[torch.cuda.FloatTensor of size 1x4 (GPU 0)]



In [39]:
# Examine the replay buffer.
terminals = rb.filter_by(sars_filter(r=0))
print(len(terminals))

# action-value at 's' was wrong. Search the replay buffer for occurrences of
# the correct transition. It's likely missing, which led the NN to overfit
# on missing data.
trans_s = rb.filter_by(sars_filter(s=s))
trans_s_a = rb.filter_by(sars_filter(s=s,a=2))
print(f'Num with (s): {len(trans_s)}. Num with (s, a): {len(trans_s_a)}')


1701
Num with (s): 65. Num with (s, a): 43


In [35]:
env.visualize(exp.policy, steps=20)

array([[0, 0],
       [1, 0]])
array([[0, 0],
       [1, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
